In [1]:
abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
rul_models_path = ['./model/RULmodel/231023_xgb_ss_df_fl.pickle','./model/RULmodel/231023_xgb_ss_df_pb.pickle','./model/RULmodel/231023_xgb_ss_df_fl.pickle']
inverse_scalers_path = ['./model/RULmodel/231023_ss_y_df_fl.pickle','./model/RULmodel/231023_ss_y_df_pb.pickle','./model/RULmodel/231023_ss_y_df_ph.pickle']
scalers_path = ['./model/RULmodel/231023_ss_x_df_fl.pickle','./model/RULmodel/231023_ss_x_df_pb.pickle','./model/RULmodel/231023_ss_x_df_ph.pickle']

In [3]:
import joblib

def load_object_with_joblib(path):
    return joblib.load(path)

def get_model_features_count(model):
    try:
        return len(model.feature_importances_)
    except AttributeError:
        return "Not available for this model"

def get_scaler_feature_count(scaler):
    try:
        return scaler.n_features_in_
    except AttributeError:
        return "Not available for this scaler"

# 모델의 컬럼 개수 확인
for path in abnormal_models_path:
    model = load_object_with_joblib(path)
    print(f"{path} model features count: {get_model_features_count(model)}")

# 스케일러의 컬럼 개수 확인
for path in scalers_path:
    scaler = load_object_with_joblib(path)
    print(f"{path} scaler features count: {get_scaler_feature_count(scaler)}")


./model/abnormal_detect/RF_model(FL).pkl model features count: 10
./model/abnormal_detect/RF_model(PB).pkl model features count: 10
./model/abnormal_detect/RF_model(PH).pkl model features count: 10
./model/RULmodel/231023_ss_x_df_fl.pickle scaler features count: 10
./model/RULmodel/231023_ss_x_df_pb.pickle scaler features count: 10
./model/RULmodel/231023_ss_x_df_ph.pickle scaler features count: 9


In [84]:
import pymysql
import pickle
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import warnings
from datetime import datetime
import joblib

# XGBoost 관련 경고 숨기기
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# 현재 시간 가져오기
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 길이 기준 설정
avg_len = 500

def fetch_recent_logs(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT, ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT,
            FLOWCOOLFLOWRATE, FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
            FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
            ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER, ETCHAUX2SOURCETIMER,
            ACTUALSTEPDURATION FROM input_data_1 ORDER BY input_time DESC LIMIT {length}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

def fetch_recent_logs_for_multi(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT ACTUALROTATIONANGLE, ACTUALSTEPDURATION, ETCHBEAMCURRENT, ETCHGASCHANNEL1READBACK, 
              ETCHPBNGASREADBACK, ETCHSOURCEUSAGE, FIXTURETILTANGLE, FLOWCOOLFLOWRATE, FLOWCOOLPRESSURE, 
              IONGAUGEPRESSURE FROM input_data_1 ORDER BY input_time DESC LIMIT {length}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

def dict_to_array(data):
    """사전 형태의 데이터를 2차원 넘파이 배열로 변환하는 함수"""
    return np.array([list(item.values()) for item in data])


def predict_with_xgb_model(data,model_path,inverse_scaler_path):
    """xgboost 모델을 사용해 예측하는 함수"""
    # 데이터 형태 변환
    transformed_data = dict_to_array(data)

    """모델 예측 전 전처리 함수"""
    # 모델 불러오기
    scaler = joblib.load('./model/rul/X_scaler.pkl')
    
    # scaler 적용
    scaled_data = scaler.transform(transformed_data)

    """xgboost 모델을 사용해 예측하는 함수"""
    # 모델 불러오기
    model =  joblib.load(model_path)

    # 예측 실행
    predictions_scaled = model.predict(scaled_data)

    #### rul inverse transfomr 적용 필요!!!!
     # 모델 불러오기
    inverse_scaler = joblib.load(inverse_scaler_path)
    
    predictions = inverse_scaler.inverse_transform(predictions_scaled.reshape(-1,1))

    return predictions[:,0]

def predict_with_xgb_multi_model(data,model_path):
    """xgboost 다중분류 모델을 사용해 예측하는 함수"""
    # 데이터 형태 변환
    transformed_data = dict_to_array(data)

    """모델 예측 전 전처리 함수"""
    # 모델 불러오기
    scaler = joblib.load('./model/abnormal_detect/StandardScaler.pkl')
    
    # scaler 적용
    scaled_data = scaler.transform(transformed_data)

    """xgboost 모델을 사용해 예측하는 함수"""
    # 모델 불러오기
    model=  joblib.load(model_path) 

    # 예측 실행
    predictions = model.predict(scaled_data)

    return predictions

def compute_moving_average(data, window_size=avg_len):
    """이동평균 계산하는 함수"""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

import numpy as np

def compute_moving_median(data, window_size):
    """이동 중앙값 계산하는 함수"""
    tmp_data = data.reshape(1,-1)
    num_data = len(tmp_data)
    medians = []

    for i in range(num_data - window_size + 1):
        window_data = tmp_data[i:i+window_size]
        medians.append(np.median(window_data))

    return np.array(medians)

########################################################################################################################################
########################################################################################################################################
########################################################################################################################################

### 데이터 밀어 넣기

def insert_single_data(connection, single_data):
    try:
        with connection.cursor() as cursor:
            # 현재 시간 가져오기
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO input_data_1 (time, Tool, stage, Lot, runnum, recipe, recipe_step,
       IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT,
       ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT, FLOWCOOLFLOWRATE,
       FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
       FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
       FIXTURESHUTTERPOSITION, ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER,
       ETCHAUX2SOURCETIMER, ACTUALSTEPDURATION, input_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "{current_time}")'''
            cursor.execute(sql, single_data)
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()
    return current_time

def insert_single_rul_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO rul_1(rul_fl, rul_pb, rul_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()

def insert_single_multi_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO multi_1(multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()




def main():
    # 데이터베이스 연결 설정

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    # CSV 파일 읽기
    df = pd.read_csv("./model_data_input/test.csv")

    # DataFrame에서 튜플 리스트로 데이터 변환
    data_tuples = list(df.itertuples(index=False, name=None))

    abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
    rul_models_path = ['./model/rul/xgb_model_for_fl.pkl','./model/rul/xgb_model_for_pb.pkl','./model/rul/xgb_model_for_ph.pkl']
    inverse_scalers_path = ['./model/rul/y_scaler_for_fl.pkl','./model/rul/y_scaler_for_pb.pkl','./model/rul/y_scaler_for_ph.pkl']

    ## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

    for single_data in data_tuples:

        try:
            # 데이터 삽입
            current_time = insert_single_data(connection, single_data)

            data = fetch_recent_logs(length=avg_len)
            data_for_multi = fetch_recent_logs_for_multi(length=avg_len)

            rul_insert = []
            multi_insert = []

            for i in range(len(abnormal_models_path)):
            
                # 가져온 데이터를 기반으로 예측하기
                predictions = predict_with_xgb_model(data,rul_models_path[i],inverse_scalers_path[i])
                predictions_for_multi = predict_with_xgb_multi_model(data_for_multi,abnormal_models_path[i])
                
                # 이동평균 계산하기
                window_size = min(avg_len, len(predictions))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
                moving_avg = compute_moving_average(predictions, window_size=window_size)
                window_size_for_multi = min(avg_len, len(predictions_for_multi))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
                moving_avg_for_multi = compute_moving_average(predictions_for_multi, window_size=window_size_for_multi)
                
                rul_insert.append(moving_avg[0])
                print(6)
                multi_insert.append(predictions_for_multi[0])
                print(7)
                
            # pred 데이터 삽입
            insert_single_rul_data(connection, rul_insert, current_time)
            print(8)
            insert_single_multi_data(connection, multi_insert, current_time)
            print(9)

            time.sleep(4)  # 4초 대기

        except Exception as e:
            print(f"Error: {e}")
    
    # 연결 종료
    connection.close()    

main()


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


KeyboardInterrupt: 

In [75]:

# CSV 파일 읽기
df = pd.read_csv("./model_data_input/test.csv")

# DataFrame에서 튜플 리스트로 데이터 변환
data_tuples = list(df.itertuples(index=False, name=None))

abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
rul_models_path = ['./model/rul/xgb_model_for_fl.pkl','./model/rul/xgb_model_for_pb.pkl','./model/rul/xgb_model_for_ph.pkl']
inverse_scalers_path = ['./model/rul/y_scaler_for_fl.pkl','./model/rul/y_scaler_for_pb.pkl','./model/rul/y_scaler_for_ph.pkl']

## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

In [76]:
connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

In [77]:
data = fetch_recent_logs(length=avg_len)
data

[{'IONGAUGEPRESSURE': -0.0794283467426016,
  'ETCHBEAMVOLTAGE': -0.958814437672802,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHSUPPRESSORVOLTAGE': -0.94619431001018,
  'ETCHSUPPRESSORCURRENT': -0.9390426794102577,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'FIXTURETILTANGLE': 2.027847975076401,
  'ROTATIONSPEED': -0.0132071874464973,
  'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'ETCHAUXSOURCETIMER': 2.8715106903677774,
  'ETCHAUX2SOURCETIMER': 3.1759407144678846,
  'ACTUALSTEPDURATION': 4.03778468798202},
 {'IONGAUGEPRESSURE': -0.0794283467426016,
  'ETCHBEAMVOLTAGE': -0.958814437672802,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHSUPPRESSORVOLTAGE': -0.94619431001018,
  'ETCHSUPPRESSORCURRENT': -0.9390426794102577,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.332806895735074

In [78]:
predictions = predict_with_xgb_model(data,rul_models_path[0],inverse_scalers_path[0])
predictions

c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[2169449. ],
       [2169449. ],
       [2169449. ],
       [2169449. ],
       [1580659.8],
       [1541943.1],
       [1875674.9],
       [1819576.9],
       [1396611.4],
       [1735063.5],
       [1735063.5],
       [1733304.1],
       [2054692.5],
       [2056451.9],
       [1735063.5],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1875674.9],
       [1733304.1],
       [2054692.5],
       [1497973.1],
       [1258518.9],
       [1344791.4],
       [1734558.6],
       [1489876.6],
       [1624971.2],
       [1562869.8],
       [1467559.1],
       [1693362.1],
       [1693362.1],
       [1692258.5],
       [1808169.8],
       [1861418.4],
       [1692258.5],
       [1692258.5],
       [1861418.4],
       [1861418.4],
       [1808169.8],
       [1778530.9],
       [1894442.5],
       [1894442.5],
       [1809929.1],
       [1694017.9],
       [1723802.4],
       [1839713.6],
       [1839713.6],
       [1892962.2],


In [79]:
window_size = min(avg_len, len(predictions))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
window_size

398

In [80]:
predictions[:,0]

array([2169449. , 2169449. , 2169449. , 2169449. , 1580659.8, 1541943.1,
       1875674.9, 1819576.9, 1396611.4, 1735063.5, 1735063.5, 1733304.1,
       2054692.5, 2056451.9, 1735063.5, 1733304.1, 1733304.1, 1733304.1,
       1733304.1, 1733304.1, 1875674.9, 1733304.1, 2054692.5, 1497973.1,
       1258518.9, 1344791.4, 1734558.6, 1489876.6, 1624971.2, 1562869.8,
       1467559.1, 1693362.1, 1693362.1, 1692258.5, 1808169.8, 1861418.4,
       1692258.5, 1692258.5, 1861418.4, 1861418.4, 1808169.8, 1778530.9,
       1894442.5, 1894442.5, 1809929.1, 1694017.9, 1723802.4, 1839713.6,
       1839713.6, 1892962.2, 1597665. , 1809259.9, 1978703. , 1936332. ,
       1978703. , 1978703. , 1995485.5, 1995485.5, 1775048.4, 1978703. ,
       1978703. , 1978703. , 1978703. , 1978703. , 2114300.5, 2049412.1,
       1978703. , 1978703. , 2114300.5, 1910645.6, 2114300.5, 1936332. ,
       1954776. , 2116059.8, 2116059.8, 1980462.4, 1980462.4, 1950332.5,
       1950332.5, 1927738.2, 1760793.6, 1469662.6, 

In [81]:
moving_avg = compute_moving_average(predictions[:,0], window_size=window_size)
moving_avg

array([1968009.60081658])

In [26]:
current_time = insert_single_data(connection, data_tuples[0])
current_time

'2023-10-24 12:36:08'

In [27]:
data_for_multi = fetch_recent_logs_for_multi(length=avg_len)
data_for_multi

[{'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'FIXTURETILTANGLE': 2.027847975076401,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'IONGAUGEPRESSURE': -0.0794283467426016},
 {'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'FIXTURETILTANGLE': 2.027847975076401,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'IONGAUGEPRESSURE': -0.0794283467426016},
 {'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  

In [28]:
predictions_for_multi = predict_with_xgb_multi_model(data_for_multi,abnormal_models_path[0])
predictions_for_multi

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [29]:
multi_insert = []

In [30]:
multi_insert.append(predictions_for_multi[0])

In [31]:
multi_insert

[0]

In [17]:
len(data_for_multi[0].values())

10